In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
data = {'32.047691, 34.793865': [{'geometry': {'location': {'lat': 32.0852999, 'lng': 34.78176759999999}, 'viewport': {'northeast': {'lat': 32.146611, 'lng': 34.8519761}, 'southwest': {'lat': 32.029252, 'lng': 34.7425159}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png', 'name': 'Tel Aviv-Yafo', 'photos': [{'height': 3048, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116127180746491988473">ITAY ROITMAN AERIAL PRODUCTION</a>'], 'photo_reference': 'CkQ0AAAAXnbs1BvD83QuDi3DffQtAD_VQnaMCyG5syGb4lH5DOiiQqPz-9ArGz8DGKDloVhnsmtPsxbxyc7XzPEFKV6yrBIQwIPIxsb-z0HOXqh2Hj_gshoUni2L-IayiCPVsteVexVNxdvirfc', 'width': 5418}], 'place_id': 'ChIJH3w7GaZMHRURkD-WwKJy-8E', 'reference': 'ChIJH3w7GaZMHRURkD-WwKJy-8E', 'scope': 'GOOGLE', 'types': ['locality', 'political'], 'vicinity': 'Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.05023600000001, 'lng': 34.791867}, 'viewport': {'northeast': {'lat': 32.0516027802915, 'lng': 34.7932570802915}, 'southwest': {'lat': 32.0489048197085, 'lng': 34.7905591197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/pharmacy_cross-71.png', 'name': 'Peleg Hiper Pharm', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJFbUdNWpLHRUR-wO6GuhWP58', 'plus_code': {'compound_code': '3Q2R+3P Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+3P'}, 'reference': 'ChIJFbUdNWpLHRUR-wO6GuhWP58', 'scope': 'GOOGLE', 'types': ['pharmacy', 'health', 'point_of_interest', 'store', 'establishment'], 'vicinity': 'Tel Aviv-Yafo'}, {'business_status': 'CLOSED_TEMPORARILY', 'geometry': {'location': {'lat': 32.045517, 'lng': 34.8034329}, 'viewport': {'northeast': {'lat': 32.0468504302915, 'lng': 34.80466768029149}, 'southwest': {'lat': 32.0441524697085, 'lng': 34.8019697197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png', 'name': "Chen's Holiday Home: Women Only", 'permanently_closed': True, 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110375515532128103684">Alfredo M</a>'], 'photo_reference': 'CkQ0AAAAWaw8JLDW4Nh72qiNJAr-kVotJyzy21Zx3f08WZONGzhK6b0urosSIeiP4iY6SpjgHA1Jm2SgU6RFSxjObaP2fBIQ-LtMaeDz2RAaztfk9TbxNBoUil3odVXTgRSRMhNmhsF4AVBG0os', 'width': 3120}], 'place_id': 'ChIJ44RNRxZLHRUR8wC2sAnAYC4', 'plus_code': {'compound_code': '2RW3+69 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2RW3+69'}, 'rating': 4.1, 'reference': 'ChIJ44RNRxZLHRUR8wC2sAnAYC4', 'scope': 'GOOGLE', 'types': ['lodging', 'point_of_interest', 'establishment'], 'user_ratings_total': 7, 'vicinity': 'HaRav Yitshak Taib Street 34, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0532695, 'lng': 34.7909197}, 'viewport': {'northeast': {'lat': 32.05460508029149, 'lng': 34.79231933029151}, 'southwest': {'lat': 32.0519071197085, 'lng': 34.78962136970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'Naama spices', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3984, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106138190839519735650">ליאור נעמה</a>'], 'photo_reference': 'CkQ0AAAACgzeMUX-yWZR5rQYf_fWs3Ki6hIwxZ35M_5B5L6Z57fYpS1K90ISgh2aFZ5AwxEtexfMeJST9tCtTZkrynp1rRIQ1xSm49xxG_9lPw-4c7fkcxoUXZuY40qzrCq9oHnLJqeUfqBOBLM', 'width': 2988}], 'place_id': 'ChIJ1x0rB2xLHRURJGZw0Y3RoRc', 'plus_code': {'compound_code': '3Q3R+89 Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q3R+89'}, 'rating': 4.7, 'reference': 'ChIJ1x0rB2xLHRURJGZw0Y3RoRc', 'scope': 'GOOGLE', 'types': ['food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 204, 'vicinity': 'Mevaser Street 4, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0454216, 'lng': 34.7943369}, 'viewport': {'northeast': {'lat': 32.04668108029151, 'lng': 34.7956873302915}, 'southwest': {'lat': 32.04398311970851, 'lng': 34.7929893697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'פון קאבר מחסן יבואן אביזרים לסלולר בסיטונאות ויחידים', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109053790313052137299">פון קאבר מחסן יבואן אביזרים לסלולר בסיטונאות ויחידים</a>'], 'photo_reference': 'CkQ0AAAAmL37EFNgStT621oHJlpTIqZWD09zQ7bAtSCwbQvflx3GBRMAAYgVRXN5lFlItVzrm7sXwvcnsAiFu232Cp3HaxIQBAoXRbslVQRQzYLs_HBFAhoU_GPMBOURWdWJ58p048Xk8SXj5nk', 'width': 2048}], 'place_id': 'ChIJFYeUaUBLHRURl8jiRqyltiY', 'plus_code': {'compound_code': '2QWV+5P Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWV+5P'}, 'rating': 4.8, 'reference': 'ChIJFYeUaUBLHRURl8jiRqyltiY', 'scope': 'GOOGLE', 'types': ['electronics_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 182, 'vicinity': 'Malkosh Street 11, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0481402, 'lng': 34.7936891}, 'viewport': {'northeast': {'lat': 32.04948333029149, 'lng': 34.79509068029149}, 'southwest': {'lat': 32.04678536970849, 'lng': 34.7923927197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'Big-Bag', 'opening_hours': {'open_now': False}, 'place_id': 'ChIJ8_ewjmpLHRURnkvIi_dDF80', 'plus_code': {'compound_code': '2QXV+7F Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXV+7F'}, 'rating': 5, 'reference': 'ChIJ8_ewjmpLHRURnkvIi_dDF80', 'scope': 'GOOGLE', 'types': ['furniture_store', 'home_goods_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Bat Sheva Katzenelson Street 4, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0476349, 'lng': 34.7981226}, 'viewport': {'northeast': {'lat': 32.0489774802915, 'lng': 34.7995208802915}, 'southwest': {'lat': 32.0462795197085, 'lng': 34.7968229197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'Thimble Publishing Ltd.', 'opening_hours': {'open_now': False}, 'photos': [{'height': 2272, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114368039407998019218">הוצאת אצבעוני בע&quot;מ</a>'], 'photo_reference': 'CkQ0AAAAk1LMwuTUxkB-YiVnAGVXSJzQvUaj9pkfhoagQ0tvZsIiMKPTCTvM10V3ZZUsZ8TerKGZJW-CV264nLOp9LdcVhIQbpagA-g_xc8lGvhBaEr_dhoUHi-RqspJSl9m9t-hiNLhEuFqbjw', 'width': 1772}], 'place_id': 'ChIJH7T057hMHRURAFsRitKEejU', 'plus_code': {'compound_code': '2QXX+36 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXX+36'}, 'rating': 5, 'reference': 'ChIJH7T057hMHRURAFsRitKEejU', 'scope': 'GOOGLE', 'types': ['library', 'book_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Hartuv Street 26, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0506907, 'lng': 34.7941603}, 'viewport': {'northeast': {'lat': 32.05203823029149, 'lng': 34.7955287802915}, 'southwest': {'lat': 32.04934026970849, 'lng': 34.7928308197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'ARIEL HARLEV', 'place_id': 'ChIJs35eP7FLHRUR7uEyFjWClA4', 'plus_code': {'compound_code': "3Q2V+7M Giv'atayim, Israel", 'global_code': '8G4P3Q2V+7M'}, 'rating': 5, 'reference': 'ChIJs35eP7FLHRUR7uEyFjWClA4', 'scope': 'GOOGLE', 'types': ['health', 'point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': "Bat Sheva Katzenelson Street 94, Giv'atayim"}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0440165, 'lng': 34.79545230000001}, 'viewport': {'northeast': {'lat': 32.0453692302915, 'lng': 34.7968188802915}, 'southwest': {'lat': 32.0426712697085, 'lng': 34.7941209197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'אילן הובלות', 'opening_hours': {'open_now': True}, 'photos': [{'height': 960, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115932644440982427705">נהוראי לוי</a>'], 'photo_reference': 'CkQ0AAAAHy5c-WlQt5-5qM-xWcMTP8u5HZ2RXSqtUrUiK0F5JECrb_6HaVURRopFbR_5jce66gv7xFdabMhPfiT13FFbiRIQmfPmgqmuKVmQ5uEQWZVvPxoUR3YbsojB2RfjJEwx6XzM4ledCAA', 'width': 1280}], 'place_id': 'ChIJSaxyiEBLHRURIBkQzRvLFkU', 'plus_code': {'compound_code': '2QVW+J5 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVW+J5'}, 'rating': 4.3, 'reference': 'ChIJSaxyiEBLHRURIBkQzRvLFkU', 'scope': 'GOOGLE', 'types': ['moving_company', 'storage', 'point_of_interest', 'establishment'], 'user_ratings_total': 16, 'vicinity': 'Shtulim Street 50, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.049792, 'lng': 34.79211599999999}, 'viewport': {'northeast': {'lat': 32.05116338029151, 'lng': 34.79351218029149}, 'southwest': {'lat': 32.04846541970851, 'lng': 34.7908142197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png', 'name': 'Avazi', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1080, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104324435703328319013">Moshe Krinsky</a>'], 'photo_reference': 'CkQ0AAAABByaD9B4Sl6u-r20qzUkczh9rqUEVGPYdPXyTQStWO61M8kcSo1qErnEXvgb5ePgXkAiIXeJrOlCzdUaEMFuLhIQYkFomRjWA6SDT_rQNUTXARoU8f3M536l5ZQPUaETTmdHBjWER8c', 'width': 1920}], 'place_id': 'ChIJW6tZPnxLHRURHQzVXXo51eg', 'plus_code': {'compound_code': '2QXR+WR Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXR+WR'}, 'price_level': 2, 'rating': 3.7, 'reference': 'ChIJW6tZPnxLHRURHQzVXXo51eg', 'scope': 'GOOGLE', 'types': ['restaurant', 'food', 'point_of_interest', 'establishment'], 'user_ratings_total': 829, 'vicinity': 'Etsel Street 54, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0538877, 'lng': 34.7861642}, 'viewport': {'northeast': {'lat': 32.0553412302915, 'lng': 34.78750443029149}, 'southwest': {'lat': 32.0526432697085, 'lng': 34.7848064697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png', 'name': 'Newman Center', 'opening_hours': {'open_now': True}, 'photos': [{'height': 640, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104263139648803401838">Newman Center</a>'], 'photo_reference': 'CkQ0AAAA7XwJikDgYiPYaOibTrgHu7AQHOp_GwDEVwru167hM6xms6L1Oh6JnyOpSEYNS1RuL0-WkttFGT9cSRobDsiV9RIQUATecqQiAPd4-pnH1syNcRoUROiJNfhLVJYswcA1z6P23XUqAGE', 'width': 1136}], 'place_id': 'ChIJwbGFsGhLHRURLeB8avXLm6A', 'plus_code': {'compound_code': '3Q3P+HF Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q3P+HF'}, 'rating': 4.5, 'reference': 'ChIJwbGFsGhLHRURLeB8avXLm6A', 'scope': 'GOOGLE', 'types': ['university', 'point_of_interest', 'establishment'], 'user_ratings_total': 52, 'vicinity': 'Hagana Road 34, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.049643, 'lng': 34.792461}, 'viewport': {'northeast': {'lat': 32.0509708802915, 'lng': 34.7937594302915}, 'southwest': {'lat': 32.0482729197085, 'lng': 34.7910614697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'מיס דוליטל- מספרת כלבים וחתולים', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2048, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115659950876925290559">מיס דוליטל- מספרת כלבים וחתולים</a>'], 'photo_reference': 'CkQ0AAAAvxmbAGHnJ6d_wFOf-a_tFFkMcDdXxsdJ2a75-YzqTvCOdT0tw2r_u5Z6E7ytmBi8JZ6BnEJ7LSJhhPga4F3LchIQkKIrsoNCGG4_65xqcbdKahoUoOQLss86AdFIZj7-L2OXs3OGYSo', 'width': 1152}], 'place_id': 'ChIJjQj_RWpLHRUR1vLJkH6IQhM', 'plus_code': {'compound_code': '2QXR+VX Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXR+VX'}, 'rating': 4.4, 'reference': 'ChIJjQj_RWpLHRUR1vLJkH6IQhM', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 56, 'vicinity': 'Etsel Street 71, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0515504, 'lng': 34.7917767}, 'viewport': {'northeast': {'lat': 32.0528916302915, 'lng': 34.79302198029151}, 'southwest': {'lat': 32.0501936697085, 'lng': 34.79032401970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png', 'name': 'Boaz brothers', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3024, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101857218720446422369">yuval graidy</a>'], 'photo_reference': 'CkQ0AAAARORyoznCa1OxWyxO13Gu0Cbjc-evpi5jWx0YlcdH2gPOFR9h4QWCb0M-nb5gn4fZnWFyr9JIUPTVWhhMWdxmPBIQqJPAQ-vpBIuTgz56HhUz_xoUKivbBS67g-lxgNokJiaCVqydisU', 'width': 4032}], 'place_id': 'ChIJRRUGw2tLHRURvJBw_N646-M', 'plus_code': {'compound_code': '3Q2R+JP Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+JP'}, 'rating': 4.5, 'reference': 'ChIJRRUGw2tLHRURvJBw_N646-M', 'scope': 'GOOGLE', 'types': ['restaurant', 'food', 'point_of_interest', 'establishment'], 'user_ratings_total': 361, 'vicinity': 'Etsel Street 47, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0471928, 'lng': 34.7991432}, 'viewport': {'northeast': {'lat': 32.04853713029149, 'lng': 34.80052893029149}, 'southwest': {'lat': 32.04583916970849, 'lng': 34.79783096970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': "Let's Do Studio", 'opening_hours': {'open_now': False}, 'photos': [{'height': 655, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105910820507735217136">Let&#39;s Do Studio</a>'], 'photo_reference': 'CkQ0AAAA9gKQQY8ZJ-HQIygE14qgOnw5rXNnSJ-cC07rIq_M5bs9npNiB7A0ManfXKrZFlv8lbqh5z5Vo64hSicn9bk5cxIQK6Q-xu9ueRwnx86NGq6uPhoUdB9LyxAdQIcrTbn3nbhcuNvaWwI', 'width': 1000}], 'place_id': 'ChIJdYaCUgpLHRURUzKjd47YqZ8', 'plus_code': {'compound_code': '2QWX+VM Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWX+VM'}, 'reference': 'ChIJdYaCUgpLHRURUzKjd47YqZ8', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Moshe Dayan Road 2, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0435786, 'lng': 34.7955325}, 'viewport': {'northeast': {'lat': 32.04493958029149, 'lng': 34.7969326302915}, 'southwest': {'lat': 32.0422416197085, 'lng': 34.7942346697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'מגדלי קירור שרון (מס\' 2 1985) בע"מ', 'place_id': 'ChIJpU0dm0BLHRUROAWkKAlfV7k', 'plus_code': {'compound_code': '2QVW+C6 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVW+C6'}, 'rating': 4.8, 'reference': 'ChIJpU0dm0BLHRUROAWkKAlfV7k', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': 'Shtulim Street 60, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04256290000001, 'lng': 34.7949613}, 'viewport': {'northeast': {'lat': 32.0439337302915, 'lng': 34.7963370302915}, 'southwest': {'lat': 32.0412357697085, 'lng': 34.7936390697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'ביוטי נט - מוצרי שיער', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113128518718913051104">ביוטי נט - מוצרי שיער</a>'], 'photo_reference': 'CkQ0AAAA7FVODPdLVXqrckmT7gAoQ3Rpykk7kdFutfzAeEHFL7Pa9yF8xGlcGVw0-Y52wAOgxtZYWmfB9DWQ12nAkssTWBIQfRFYQBgItmLwvbK0FErGrRoUvTa3_e9iTKm9EAk-ESJS2rSlj-Y', 'width': 4608}], 'place_id': 'ChIJhcW_dL1LHRURpsNOnvWe6bo', 'plus_code': {'compound_code': '2QVV+2X Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVV+2X'}, 'rating': 5, 'reference': 'ChIJhcW_dL1LHRURpsNOnvWe6bo', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'store', 'establishment'], 'user_ratings_total': 264, 'vicinity': 'Shtulim Street 72, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0464086, 'lng': 34.7907575}, 'viewport': {'northeast': {'lat': 32.0477573302915, 'lng': 34.79211353029149}, 'southwest': {'lat': 32.0450593697085, 'lng': 34.7894155697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'כללית טיפת חלב', 'opening_hours': {'open_now': False}, 'place_id': 'ChIJnYFBiEFLHRURikDt9ZMHcRc', 'plus_code': {'compound_code': '2QWR+H8 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWR+H8'}, 'rating': 5, 'reference': 'ChIJnYFBiEFLHRURikDt9ZMHcRc', 'scope': 'GOOGLE', 'types': ['health', 'point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Tevet Street 15, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04632439999999, 'lng': 34.7936848}, 'viewport': {'northeast': {'lat': 32.0476892302915, 'lng': 34.7950329302915}, 'southwest': {'lat': 32.0449912697085, 'lng': 34.7923349697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'הובלות הקרנפים', 'place_id': 'ChIJMZILNEBLHRURdl9J5ULaxdM', 'plus_code': {'compound_code': '2QWV+GF Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWV+GF'}, 'rating': 3.6, 'reference': 'ChIJMZILNEBLHRURdl9J5ULaxdM', 'scope': 'GOOGLE', 'types': ['moving_company', 'point_of_interest', 'establishment'], 'user_ratings_total': 5, 'vicinity': 'Shibolim Street 33, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0483356, 'lng': 34.79059069999999}, 'viewport': {'northeast': {'lat': 32.0496669302915, 'lng': 34.79199053029149}, 'southwest': {'lat': 32.0469689697085, 'lng': 34.7892925697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'צוות לכל עניין שיפוצים', 'photos': [{'height': 267, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107113685323212316936">צוות לכל עניין שיפוצים</a>'], 'photo_reference': 'CkQ0AAAAZqIg6pLKKz5X0H21EMhPVrmMrZUS3bGgbxn3qeCs4sPG3ZSQk4MTvBc9xkxPAqEGZ26obRHPZJ83YAQbyTESERIQCrwclpLZ1bRKq4MlgCgaXhoUMZO7CoG643Dw2qflK1pYeY_uOY0', 'width': 400}], 'place_id': 'ChIJZTQjAGpLHRURk7HrNhx0Y5s', 'plus_code': {'compound_code': '2QXR+86 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXR+86'}, 'rating': 4, 'reference': 'ChIJZTQjAGpLHRURk7HrNhx0Y5s', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Kehilat Mexico Street 13, Tel Aviv-Yafo'}, {'geometry': {'location': {'lat': 32.048638, 'lng': 34.796509}, 'viewport': {'northeast': {'lat': 32.058095, 'lng': 34.8125164}, 'southwest': {'lat': 32.03918, 'lng': 34.7805016}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/geocode-71.png', 'name': "Beit Ya'akov", 'place_id': 'ChIJcZThKxVLHRURudHXDlib5jo', 'reference': 'ChIJcZThKxVLHRURudHXDlib5jo', 'scope': 'GOOGLE', 'types': ['neighborhood', 'political'], 'vicinity': 'Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.045512, 'lng': 34.7896017}, 'viewport': {'northeast': {'lat': 32.0468668802915, 'lng': 34.7910032802915}, 'southwest': {'lat': 32.0441689197085, 'lng': 34.7883053197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'עזר הנגר', 'place_id': 'ChIJozQ6z0NLHRURSJo0B_24Hco', 'plus_code': {'compound_code': '2QWQ+6R Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWQ+6R'}, 'reference': 'ChIJozQ6z0NLHRURSJo0B_24Hco', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Tishrei Street 32, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0472775, 'lng': 34.7979356}, 'viewport': {'northeast': {'lat': 32.0485749802915, 'lng': 34.7992718802915}, 'southwest': {'lat': 32.0458770197085, 'lng': 34.7965739197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'מחסן עצים א.י. זוהר מוצרי עץ בע"מ', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113440624352041957566">Roy Stoopel</a>'], 'photo_reference': 'CkQ0AAAAmN5s3cqJXQr2r5b7VSkyS4lEwuiJUFD0pjJAub3TDFM-_OM-A55s96WgJuGgcbuN07-NRO07cp-QqNEXSe6XgRIQ-M2N--mUEGM1qcojXco1VhoUwHeZQpyP0tJgmvVeRtXwuRyqDSc', 'width': 3840}], 'place_id': 'ChIJT_nIxR5LHRURaYjNdE9sPSA', 'plus_code': {'compound_code': '2QWX+W5 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWX+W5'}, 'rating': 4.8, 'reference': 'ChIJT_nIxR5LHRURaYjNdE9sPSA', 'scope': 'GOOGLE', 'types': ['hardware_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 121, 'vicinity': 'Lehi Road 95, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04613269999999, 'lng': 34.7956838}, 'viewport': {'northeast': {'lat': 32.0474788302915, 'lng': 34.7970162802915}, 'southwest': {'lat': 32.0447808697085, 'lng': 34.7943183197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'אמבט ליין בע"מ', 'place_id': 'ChIJs7xuBUBLHRURDw6CsFvz19U', 'plus_code': {'compound_code': '2QWW+F7 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWW+F7'}, 'rating': 2.5, 'reference': 'ChIJs7xuBUBLHRURDw6CsFvz19U', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 14, 'vicinity': 'א.ת, נתיבות'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.05308900000001, 'lng': 34.79124000000001}, 'viewport': {'northeast': {'lat': 32.0544503302915, 'lng': 34.7926542302915}, 'southwest': {'lat': 32.0517523697085, 'lng': 34.78995626970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'גולספורט', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4032, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109537721436144157381">Matan Gabso</a>'], 'photo_reference': 'CkQ0AAAAwnlUAA0PzsuzcME4GegDPSQE2p3f5YSru18hAjUNBP9uNGuys0RdG2EgX5_M8pyoteTMBaddaUsVxBhSqD-KAhIQCbURiZ3L3VBm-wbUkzCtCxoUG2GJVcidSvvs826UelB4uHwmu9o', 'width': 3024}], 'place_id': 'ChIJnet3CGxLHRURMc6L_immLU0', 'plus_code': {'compound_code': '3Q3R+6F Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q3R+6F'}, 'rating': 4.6, 'reference': 'ChIJnet3CGxLHRURMc6L_immLU0', 'scope': 'GOOGLE', 'types': ['shoe_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 40, 'vicinity': 'Etsel Street 12, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0431292, 'lng': 34.7911632}, 'viewport': {'northeast': {'lat': 32.0445118302915, 'lng': 34.7924027802915}, 'southwest': {'lat': 32.04181386970851, 'lng': 34.7897048197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'סמילנסקי סגנון חיים מזון ומשקאות', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104828539453519464082">סמילנסקי סגנון חיים מזון ומשקאות</a>'], 'photo_reference': 'CkQ0AAAAEm2y2n3aSI8B8ALX887ZQV9cWGzP8MwQBHOxlki-cX1YRRiOQcaTJBRRM9e-Y3l3InY_5YoNyliaumyYwKrYlhIQMPIrTHJ8FeqNlPQgVyLcsxoUZLmbu4Huu6AKajw_0H-kN8t07Cw', 'width': 2048}], 'place_id': 'ChIJS89KU0FLHRURhQdE1ImUgA8', 'plus_code': {'compound_code': '2QVR+7F Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVR+7F'}, 'rating': 3, 'reference': 'ChIJS89KU0FLHRURhQdE1ImUgA8', 'scope': 'GOOGLE', 'types': ['food', 'point_of_interest', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'Avenue Street 11, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.046315, 'lng': 34.79201949999999}, 'viewport': {'northeast': {'lat': 32.0476689302915, 'lng': 34.7932968302915}, 'southwest': {'lat': 32.0449709697085, 'lng': 34.7905988697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/restaurant-71.png', 'name': 'האופה מבגדד', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2976, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117057492156967749228">nir766 nir766</a>'], 'photo_reference': 'CkQ0AAAA40DhTsylPyhkDNUwNRM60HblwGqYYE1C74qfhFeEut4tJoTmO4Hl2ONrMKl-UMfi3LOfq-UHKLLH_fw-K3_W_xIQkEFOq8VBia_74kRCoDaiJhoUjNqulmftI3XLGR1UDCzGkQvmMLM', 'width': 2976}], 'place_id': 'ChIJebKwlUFLHRUR9AiA9en38HA', 'plus_code': {'compound_code': '2QWR+GR Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWR+GR'}, 'price_level': 1, 'rating': 4.3, 'reference': 'ChIJebKwlUFLHRUR9AiA9en38HA', 'scope': 'GOOGLE', 'types': ['bakery', 'cafe', 'restaurant', 'food', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 1956, 'vicinity': 'Herutenu Street 11, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.05154729999999, 'lng': 34.7927607}, 'viewport': {'northeast': {'lat': 32.0528881802915, 'lng': 34.7941600802915}, 'southwest': {'lat': 32.0501902197085, 'lng': 34.7914621197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'הכל לתלמיד', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4160, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111914149267311405592">daniel 23</a>'], 'photo_reference': 'CkQ0AAAAO8josVKuSCoB8aE8RRkDhvfhqIjyU3vi0TyWogCbjCpd3aOrS1joe2o02bL5MkuxFgBO4IotHAY_Z-jffZsxSRIQk5Dx6uqUEM8a8u4D1OWbLBoUYiNYRzaqjuD1lfRR_jguTFi9GKA', 'width': 2340}], 'place_id': 'ChIJ4V-amWtLHRURX1fMGQf8enM', 'plus_code': {'compound_code': '3Q2V+J4 Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2V+J4'}, 'rating': 4.4, 'reference': 'ChIJ4V-amWtLHRURX1fMGQf8enM', 'scope': 'GOOGLE', 'types': ['book_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 46, 'vicinity': "Yekhi'am Street 38, Tel Aviv-Yafo"}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04882119999999, 'lng': 34.7991652}, 'viewport': {'northeast': {'lat': 32.0501625302915, 'lng': 34.8005663802915}, 'southwest': {'lat': 32.0474645697085, 'lng': 34.7978684197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'אמבולנס נועם בע"מ', 'place_id': 'ChIJHXyCWRRLHRURD2wy9k1qIQo', 'plus_code': {'compound_code': '2QXX+GM Ramat Gan, Israel', 'global_code': '8G4P2QXX+GM'}, 'rating': 4.5, 'reference': 'ChIJHXyCWRRLHRURD2wy9k1qIQo', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Moshe Dayan Road 23, Ramat Gan'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.047958, 'lng': 34.793377}, 'viewport': {'northeast': {'lat': 32.04934298029151, 'lng': 34.79476073029149}, 'southwest': {'lat': 32.04664501970851, 'lng': 34.7920627697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'ישראל מוכתר סוכנות ביטוח', 'opening_hours': {'open_now': False}, 'place_id': 'ChIJH7T057hMHRURBYNVsLvrkW4', 'plus_code': {'compound_code': '2QXV+59 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXV+59'}, 'rating': 5, 'reference': 'ChIJH7T057hMHRURBYNVsLvrkW4', 'scope': 'GOOGLE', 'types': ['insurance_agency', 'point_of_interest', 'establishment'], 'user_ratings_total': 5, 'vicinity': 'Etsel Street 76, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04746899999999, 'lng': 34.79304800000001}, 'viewport': {'northeast': {'lat': 32.0487866302915, 'lng': 34.79440278029151}, 'southwest': {'lat': 32.0460886697085, 'lng': 34.79170481970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'כלבלב על הגב', 'opening_hours': {'open_now': True}, 'photos': [{'height': 290, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107067859302582835434">כלבלב על הגב</a>'], 'photo_reference': 'CkQzAAAA4QfzuzRUsFBlNIn4gGd4q5uiYDhHOVKfYT_Bm5cQpAoNMiw6Qw4v7uxB_TUp1hSEYKP59qR-tnTPolCHqI63nBIQ1Zrlee9rltigFz2CpF8pkhoUuk_R5VG41TkvhKjOj96NKM5imnU', 'width': 870}], 'place_id': 'ChIJ78App2RLHRURYCp5_L0T7HI', 'plus_code': {'compound_code': '2QWV+X6 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWV+X6'}, 'rating': 3.7, 'reference': 'ChIJ78App2RLHRURYCp5_L0T7HI', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'לח״י, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0483844, 'lng': 34.7927663}, 'viewport': {'northeast': {'lat': 32.0497310302915, 'lng': 34.7941523302915}, 'southwest': {'lat': 32.0470330697085, 'lng': 34.7914543697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'בן שמעון פלוריש בע"מ', 'place_id': 'ChIJgRqQZWpLHRURJ-iyYhLwBhc', 'reference': 'ChIJgRqQZWpLHRURJ-iyYhLwBhc', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Tradyon Street, תרדיון'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0483844, 'lng': 34.7927663}, 'viewport': {'northeast': {'lat': 32.0497310302915, 'lng': 34.7941523302915}, 'southwest': {'lat': 32.0470330697085, 'lng': 34.7914543697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'היי-טקס מיסודה של תפרון בע"מ', 'place_id': 'ChIJgRqQZWpLHRURxWTpcTtDvg4', 'reference': 'ChIJgRqQZWpLHRURxWTpcTtDvg4', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Tradyon Street, תרדיון'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0492411, 'lng': 34.7927679}, 'viewport': {'northeast': {'lat': 32.0505560302915, 'lng': 34.7940403302915}, 'southwest': {'lat': 32.0478580697085, 'lng': 34.7913423697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'Leumit Health Care Services', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116908678282579672212">Michael Berrebi</a>'], 'photo_reference': 'CkQ0AAAAFsBmwRHVPqZtzD_Jxll2Ni8GinyG0IxAOenqfCTUqYYYS2D_LiLXT16ZJR8kw2qkELhoiTnKJw18xZ9UBp2RjRIQvzC3ZUTQKEg9I2HGGw5wNRoUm057qVHcoHI2a1_Q-bYFJ7xLeIk', 'width': 5312}], 'place_id': 'ChIJwfBKQ2pLHRURixgq4y2ErLY', 'plus_code': {'compound_code': '2QXV+M4 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXV+M4'}, 'rating': 5, 'reference': 'ChIJwfBKQ2pLHRURixgq4y2ErLY', 'scope': 'GOOGLE', 'types': ['health', 'point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': 'Etsel Street 79, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0494884, 'lng': 34.795002}, 'viewport': {'northeast': {'lat': 32.0508398802915, 'lng': 34.7962923802915}, 'southwest': {'lat': 32.0481419197085, 'lng': 34.7935944197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/doctor-71.png', 'name': 'United Hatzalah Tel Aviv - Volunteers Center', 'photos': [{'height': 640, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105193815577242357605">David Leigh</a>'], 'photo_reference': 'CkQ0AAAAdFLxglp18wh1mTPqZkVmTD2BkMB-KhdFE1ObeiI2Fc100UueV1M3qu4laDHi4mYnhk5rHyuCKWys9BGdH7dFaBIQCRak4tTht5uVz6q14mT49hoUT0tlaqwHD2c8aPNH3b2riRXuPDQ', 'width': 640}], 'place_id': 'ChIJ_T37IGtLHRURL7TnvhlRhEI', 'plus_code': {'compound_code': '2QXW+Q2 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXW+Q2'}, 'reference': 'ChIJ_T37IGtLHRURL7TnvhlRhEI', 'scope': 'GOOGLE', 'types': ['hospital', 'health', 'point_of_interest', 'establishment'], 'vicinity': "Bo'az Street 24, Tel Aviv-Yafo"}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0495584, 'lng': 34.7949658}, 'viewport': {'northeast': {'lat': 32.0509090302915, 'lng': 34.7962762802915}, 'southwest': {'lat': 32.0482110697085, 'lng': 34.7935783197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'בית המתנדב איחוד הצלה תל אביב', 'photos': [{'height': 3695, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115711847913597144099">ישי דוד תורג&#39;מן</a>'], 'photo_reference': 'CkQ0AAAAUencLyH6ZYOrED7ZoAf1h5xW9n1AwonV6UFd6KWkUwl2hFjgw-aGALcHDy7yb6MKVsN7RNNxHG1vIggxynu2AhIQFJM_LiVIxjmpf0g-wKN54hoUim2kvLnIxxBKYZakfVZ9Ti6Hir8', 'width': 6149}], 'place_id': 'ChIJNbC5IWtLHRURqsLMTEj2YiA', 'plus_code': {'compound_code': '2QXV+RX Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXV+RX'}, 'rating': 4.7, 'reference': 'ChIJNbC5IWtLHRURqsLMTEj2YiA', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 7, 'vicinity': "Bo'az Street 24, Tel Aviv-Yafo"}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04613269999999, 'lng': 34.7956838}, 'viewport': {'northeast': {'lat': 32.0474788302915, 'lng': 34.7970162802915}, 'southwest': {'lat': 32.0447808697085, 'lng': 34.7943183197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'פנל מטבחים א. כהן בע"מ', 'place_id': 'ChIJs7xuBUBLHRURukTM3TPg9Xk', 'plus_code': {'compound_code': '2QWW+F7 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWW+F7'}, 'rating': 3.3, 'reference': 'ChIJs7xuBUBLHRURukTM3TPg9Xk', 'scope': 'GOOGLE', 'types': ['home_goods_store', 'general_contractor', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'א.ת, נתיבות'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0475917, 'lng': 34.7964774}, 'viewport': {'northeast': {'lat': 32.0488595302915, 'lng': 34.79780678029149}, 'southwest': {'lat': 32.0461615697085, 'lng': 34.7951088197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png', 'name': 'Yosef Chaim Yeshiva', 'place_id': 'ChIJkSRtSxVLHRUR-O2_hYRe-2g', 'plus_code': {'compound_code': '2QXW+2H Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXW+2H'}, 'rating': 5, 'reference': 'ChIJkSRtSxVLHRUR-O2_hYRe-2g', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Lehi Road 79, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04526, 'lng': 34.79432399999999}, 'viewport': {'northeast': {'lat': 32.0466228802915, 'lng': 34.7956727802915}, 'southwest': {'lat': 32.0439249197085, 'lng': 34.7929748197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'Cover for iPhone 7', 'opening_hours': {'open_now': True}, 'photos': [{'height': 2988, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115746620762884128337">פון קאבר</a>'], 'photo_reference': 'CkQ0AAAA4wjhntwwnm63jUZQO7NGOxKk4yA_tQfGhGo47Pele2quj6CxiDVKOIzre-LmbubCsXU7St5sEGwLOaXqYjqjrRIQjEyDIXfygjrtwyZIggnCwRoUO-FVi6tJUmQPQohGwabjO3opWZU', 'width': 5312}], 'place_id': 'ChIJE5PEREBLHRUR20Dn8NnwUI8', 'plus_code': {'compound_code': '2QWV+4P Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWV+4P'}, 'rating': 4.6, 'reference': 'ChIJE5PEREBLHRUR20Dn8NnwUI8', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 11, 'vicinity': 'Malkosh Street 15, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0427869, 'lng': 34.7948179}, 'viewport': {'northeast': {'lat': 32.04406528029151, 'lng': 34.7961375302915}, 'southwest': {'lat': 32.04136731970851, 'lng': 34.7934395697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'מיטל עיצובים - ציוד למספרות', 'opening_hours': {'open_now': False}, 'photos': [{'height': 540, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116245164755109652441">מיטל עיצובים - ציוד למספרות</a>'], 'photo_reference': 'CkQ0AAAADSwLeGwxNy5mgb9ppoPQ-BGbQni3Q_2NDZq09uv7gy4wwFbCrqcI9hM74e1p78Kga2x6Tle31b6mnb7X9cmTrBIQYq7CGl8HLqAKsvr5YbrrhBoUyhPICnJSfYVZhp_yjJIgACsB44k', 'width': 720}], 'place_id': 'ChIJFSteoL1LHRUR6u2CN6bXQ_g', 'plus_code': {'compound_code': '2QVV+4W Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVV+4W'}, 'rating': 4.8, 'reference': 'ChIJFSteoL1LHRUR6u2CN6bXQ_g', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'store', 'establishment'], 'user_ratings_total': 125, 'vicinity': 'Shtulim Street 70, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0452517, 'lng': 34.7951346}, 'viewport': {'northeast': {'lat': 32.0465937802915, 'lng': 34.79644733029149}, 'southwest': {'lat': 32.0438958197085, 'lng': 34.7937493697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'אביזרים לסלולר נוט 20 אולטרה', 'opening_hours': {'open_now': True}, 'photos': [{'height': 642, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110802681359596399292">אביזרים לסלולר נוט 20 אולטרה</a>'], 'photo_reference': 'CkQ0AAAAcN6K8NJ96Wx7hzrg-BlWUar4_DUawIDsD_wdvlKdvLyRb_2jAC4_PKUIorZ0XPTBxhqeng4RGAjOONim69ib6BIQYlh4bCBA00i4Ssnilq5PRBoU4z7HtjC5BoWfoqeSywLsNxLbxw0', 'width': 701}], 'place_id': 'ChIJaThzaUBLHRURP3JPwFrt4Y8', 'plus_code': {'compound_code': '2QWW+43 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWW+43'}, 'rating': 5, 'reference': 'ChIJaThzaUBLHRURP3JPwFrt4Y8', 'scope': 'GOOGLE', 'types': ['electronics_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'Shtulim Street, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0501729, 'lng': 34.7924121}, 'viewport': {'northeast': {'lat': 32.0515236802915, 'lng': 34.7937428802915}, 'southwest': {'lat': 32.0488257197085, 'lng': 34.7910449197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png', 'name': 'עמותת "אחריי!"', 'photos': [{'height': 1000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113118835176343914962">עמותת &quot;אחריי!&quot;</a>'], 'photo_reference': 'CkQ0AAAATSsVJv2R7Wr2GPgchLejNdx-EIJc6W0zGffptNk_TR4bVKT2wmKxLJoP1hhsQNrbCCGPa9Q27QKYge4ydzeo3RIQqZaV3OZ346_Hf3WwCqq7yxoUpgdR5y4IwXm_iyKjwpnsUTd0v7I', 'width': 1500}], 'place_id': 'ChIJR-pSs2tLHRURDJIJTPP0piA', 'plus_code': {'compound_code': '3Q2R+3X Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+3X'}, 'rating': 4.5, 'reference': 'ChIJR-pSs2tLHRURDJIJTPP0piA', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 6, 'vicinity': 'Avital Street 7, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0498434, 'lng': 34.7917095}, 'viewport': {'northeast': {'lat': 32.0511919802915, 'lng': 34.7930625302915}, 'southwest': {'lat': 32.0484940197085, 'lng': 34.7903645697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'BMC ELECTRONICS בי אם סי אלקטרוניקס', 'opening_hours': {'open_now': True}, 'photos': [{'height': 720, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/118401378259774522668">BMC ELECTRONICS בי אם סי אלקטרוניקס</a>'], 'photo_reference': 'CkQ0AAAA9aANyIM-YSABA_a6Tys0diMdhcDoljrGd6vFZ5wjQSINweOXgOH5YVJc0xfMJi1-O9jISc57i8WL4Ghu17nlGBIQtNs6l9KeSXgusujrpca6uBoUCBaJxA-MhQPBfKbxTLpvaUuCuYQ', 'width': 720}], 'place_id': 'ChIJJxs2N2pLHRURdOcvHg5O7Rs', 'plus_code': {'compound_code': '2QXR+WM Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXR+WM'}, 'rating': 5, 'reference': 'ChIJJxs2N2pLHRURdOcvHg5O7Rs', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 3, 'vicinity': 'Roni Street 9, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.050079, 'lng': 34.791994}, 'viewport': {'northeast': {'lat': 32.0514341302915, 'lng': 34.7933587802915}, 'southwest': {'lat': 32.0487361697085, 'lng': 34.7906608197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'Yoram religious articles', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1200, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/109557043974529382331">תשמישי קדושה של יורם</a>'], 'photo_reference': 'CkQ0AAAAGkP1pMZh-mMClEMK24bG02kSrPQZRsdijrlfQx-X56IUziSStX7pr17DT2lPLhlySxn1rxWl8Jum0bqhMcfXQRIQbwL3NhxzxUp-SuVxtLBkhRoUjwYFGmcjZZ_HDiwraasQcrC6ug4', 'width': 1600}], 'place_id': 'ChIJGxzTymtLHRURYU4riQUt9Ak', 'plus_code': {'compound_code': '3Q2R+2Q Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+2Q'}, 'rating': 4.9, 'reference': 'ChIJGxzTymtLHRURYU4riQUt9Ak', 'scope': 'GOOGLE', 'types': ['book_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 207, 'vicinity': 'Etsel Street 52, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0501585, 'lng': 34.7920318}, 'viewport': {'northeast': {'lat': 32.05151393029149, 'lng': 34.7933975302915}, 'southwest': {'lat': 32.0488159697085, 'lng': 34.7906995697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'דוד פלטין מעצב שיער - David Platin', 'opening_hours': {'open_now': True}, 'photos': [{'height': 720, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108018466389697722792">דוד פלטין מעצב שיער - David Platin</a>'], 'photo_reference': 'CkQ0AAAAusuYV7Yf92CAmchvAWnvWLsPJqZbS6JTYKnKSz-u9i9KaNr39kSGv54823z_W6xUx9SifoG4wJstEWzdedM2XBIQaRih_dzqJtjto85HqAxlsBoUClI9bSmAYMbhcO5A2B7EvkfwbrA', 'width': 960}], 'place_id': 'ChIJ83uCSmpLHRUR5ywJi6DT1KA', 'plus_code': {'compound_code': '3Q2R+3R Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+3R'}, 'rating': 4.6, 'reference': 'ChIJ83uCSmpLHRUR5ywJi6DT1KA', 'scope': 'GOOGLE', 'types': ['hair_care', 'health', 'point_of_interest', 'establishment'], 'user_ratings_total': 32, 'vicinity': 'Etsel Street 67, Tel Aviv-Yafo'}, {'business_status': 'CLOSED_TEMPORARILY', 'geometry': {'location': {'lat': 32.0448284, 'lng': 34.79544999999999}, 'viewport': {'northeast': {'lat': 32.0461604802915, 'lng': 34.7967151302915}, 'southwest': {'lat': 32.04346251970851, 'lng': 34.7940171697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'Hakvutza be yaffo', 'permanently_closed': True, 'photos': [{'height': 3264, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/105427459688129596887">Orit Misrahi</a>'], 'photo_reference': 'CkQ0AAAAkkhX5rgPoxzYLp3Dg8y0MG1Hm8dU7Rdr9fieeV9WO1m0lDTQElPKilMrc74l8maQhuJ1vFur1eee_QI6NNmtzhIQ6wHPycW8SdOplh2838fs2xoUq3QxDlJo_0z7Wlvau7eOG4SiEFY', 'width': 2448}], 'place_id': 'ChIJLQClZUBLHRURTlDI9Uxx7gs', 'plus_code': {'compound_code': '2QVW+W5 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVW+W5'}, 'rating': 4, 'reference': 'ChIJLQClZUBLHRURTlDI9Uxx7gs', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 23, 'vicinity': 'Shtulim Street 47, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04630959999999, 'lng': 34.7904963}, 'viewport': {'northeast': {'lat': 32.0476579802915, 'lng': 34.7918452302915}, 'southwest': {'lat': 32.0449600197085, 'lng': 34.7891472697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'שיעורי מחשבים, אינטרנט וסמארטפונים', 'opening_hours': {'open_now': False}, 'photos': [{'height': 315, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116195328218540759620">שיעורי מחשבים, אינטרנט וסמארטפונים</a>'], 'photo_reference': 'CkQ0AAAAbuz3skpl00cqbaM16tX06Uva12r2aJhjwBTwgOVbbt5ISCyKOIVpP2JyCuxJ_nMERgiDuSAmst-_edBm5bspABIQlwUqtGefHB4ara6H95bJLxoUMwkxfZWgCC65C0yyjN4iRDWy8kE', 'width': 851}], 'place_id': 'ChIJH5unc0FLHRURf2b1BaKRClU', 'plus_code': {'compound_code': '2QWR+G5 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWR+G5'}, 'reference': 'ChIJH5unc0FLHRURf2b1BaKRClU', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'vicinity': 'Tevet Street 14, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0447415, 'lng': 34.79522550000001}, 'viewport': {'northeast': {'lat': 32.04609688029149, 'lng': 34.79660623029149}, 'southwest': {'lat': 32.04339891970849, 'lng': 34.7939082697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'כל במה בע"מ', 'photos': [{'height': 4016, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115030162601688647718">Mark Burshtein</a>'], 'photo_reference': 'CkQ0AAAA-MQvO33TPxzadIacnraVoS4SGxSEWckv-cCKoGYcv4BVlCAV2CARuQhy_YWrfUQs36tBYqiXA1hy1ky1xdrfXBIQI3BIok2G8uKx65vTa3jWdRoU-Uf7PEv5Qf11KuI1sg0O0CZPseE', 'width': 3008}], 'place_id': 'ChIJhxBaZEBLHRURWs93ooJDPV0', 'plus_code': {'compound_code': '2QVW+V3 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVW+V3'}, 'rating': 5, 'reference': 'ChIJhxBaZEBLHRURWs93ooJDPV0', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 2, 'vicinity': 'Shtulim Street 36, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0477262, 'lng': 34.7980681}, 'viewport': {'northeast': {'lat': 32.0490643302915, 'lng': 34.79950103029149}, 'southwest': {'lat': 32.0463663697085, 'lng': 34.7968030697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'ג\'קי בכר הפקות בע"מ', 'opening_hours': {'open_now': False}, 'place_id': 'ChIJay7rc4dLHRUR9nFiDtG_ZLk', 'plus_code': {'compound_code': '2QXX+36 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXX+36'}, 'rating': 4.8, 'reference': 'ChIJay7rc4dLHRUR9nFiDtG_ZLk', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 10, 'vicinity': 'Hartuv Street 24, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.04741119999999, 'lng': 34.78941529999999}, 'viewport': {'northeast': {'lat': 32.04875808029151, 'lng': 34.7907642302915}, 'southwest': {'lat': 32.04606011970851, 'lng': 34.7880662697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'Telecom Experts - פתרונות תקשורת ומחשוב ענן', 'opening_hours': {'open_now': True}, 'photos': [{'height': 356, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100352768249934273660">Telecom Experts - פתרונות תקשורת ומחשוב ענן</a>'], 'photo_reference': 'CkQ0AAAAd2r82xIKXGNTvaURi0boiZDhV-YZxI7_JHYKeaRC679O0brBI7u2_51ri3JrqFAcSFf_uO9y684OvNYRK5oI9hIQNlr9w6NCRra8mM1mcXVCexoUozGkAcOOtPpesnayR6Gl8rDrdPA', 'width': 540}], 'place_id': 'ChIJr4IMFUJLHRURsF0PznXyds0', 'plus_code': {'compound_code': '2QWQ+XQ Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QWQ+XQ'}, 'rating': 4.9, 'reference': 'ChIJr4IMFUJLHRURsF0PznXyds0', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 12, 'vicinity': 'Lehi Road 7, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.049439, 'lng': 34.7978518}, 'viewport': {'northeast': {'lat': 32.0507912302915, 'lng': 34.7991824802915}, 'southwest': {'lat': 32.0480932697085, 'lng': 34.7964845197085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'ברזילי מישאל', 'place_id': 'ChIJSUXl6xRLHRUR37pWa1i55cw', 'plus_code': {'compound_code': '2QXX+Q4 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXX+Q4'}, 'rating': 5, 'reference': 'ChIJSUXl6xRLHRUR37pWa1i55cw', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 1, 'vicinity': 'Plugot Street 19, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0511743, 'lng': 34.791638}, 'viewport': {'northeast': {'lat': 32.0525267302915, 'lng': 34.7930048302915}, 'southwest': {'lat': 32.0498287697085, 'lng': 34.7903068697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'סופט מובייל', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3456, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106636559571982040878">סופט מובייל</a>'], 'photo_reference': 'CkQ0AAAAC5XZ7G-76j7UlhMf_bBilkIKbFjB6dVo1deJZT5OrXdAg0zeHoAe04UP9Hk9iJa_PLlGO9KTvOX0YRnFDF7L-xIQCUhkWuX26Dau3_ZezVk6oRoU0-5wvBX6qU1yMXz1RwU-sh-F-Yk', 'width': 4608}], 'place_id': 'ChIJT5PNxWtLHRUR-tLfpKrrFa0', 'plus_code': {'compound_code': '3Q2R+FM Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+FM'}, 'rating': 4.7, 'reference': 'ChIJT5PNxWtLHRUR-tLfpKrrFa0', 'scope': 'GOOGLE', 'types': ['electronics_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 13, 'vicinity': 'Etsel Street 51, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0436585, 'lng': 34.79332000000001}, 'viewport': {'northeast': {'lat': 32.0449916302915, 'lng': 34.7946685302915}, 'southwest': {'lat': 32.04229366970851, 'lng': 34.7919705697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'סטודיו דרור כץ', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1072, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102825715641437168562">סטודיו דרור כץ</a>'], 'photo_reference': 'CkQ0AAAAyAyQ_R1uW2_Xvi5HgyowboPqQYo8LdDyIFGSdGAHZW521A0b1CJPH5TCfbN35mTP_ZJG0_2R38XM-abx-oFXNBIQm4_zZ2KdUyTAcHRXSyRcnhoUbz0gvnOtVrgOuM_SLg4UKRmFG4w', 'width': 2187}], 'place_id': 'ChIJ_d6KH0FLHRUROkiAGf8oQpQ', 'plus_code': {'compound_code': '2QVV+F8 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVV+F8'}, 'rating': 4.8, 'reference': 'ChIJ_d6KH0FLHRUROkiAGf8oQpQ', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 4, 'vicinity': 'Dokhan Street 20, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0436644, 'lng': 34.7930329}, 'viewport': {'northeast': {'lat': 32.0449977802915, 'lng': 34.7943814302915}, 'southwest': {'lat': 32.0422998197085, 'lng': 34.7916834697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'HEMERTON G remodeling and renovation of elevators', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/101450082777395515950">המרטון ג&#39;י שיפוץ וחידוש מעליות</a>'], 'photo_reference': 'CkQzAAAALVDaWmqewz91HnKmB77dcv5XrVex_Onv1TVrPGbhUD9-UGwDSfXCF2BASHQtlDoSO-lBRp3JOC9ksihEIsSBqBIQPeQdRR9SfmC5FvDIh8nMyRoUzgoa2DBI_URSIUkwtmW46jO-j1o', 'width': 2048}], 'place_id': 'ChIJRWIYIkFLHRURfckm2F2TkR0', 'plus_code': {'compound_code': '2QVV+F6 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVV+F6'}, 'rating': 5, 'reference': 'ChIJRWIYIkFLHRURfckm2F2TkR0', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 8, 'vicinity': 'Dokhan Street 14, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.051395, 'lng': 34.79124429999999}, 'viewport': {'northeast': {'lat': 32.05278888029149, 'lng': 34.79260313029149}, 'southwest': {'lat': 32.0500909197085, 'lng': 34.78990516970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'Products Center', 'opening_hours': {'open_now': False}, 'photos': [{'height': 3758, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110778540092775978245">אורן ישי</a>'], 'photo_reference': 'CkQ0AAAAMUxURE38vjVtoM1WLjnxhqcLMQiWFX69O98sS-p9RTwUMnxTsxI65Y_Xcql0pUk43lJC5L33-xjdUwxbLy__OhIQjtg26aGAhfkjd3Nw2RW17RoUR-QvVyME16YuxmdOrQ0tcEKhF9w', 'width': 2892}], 'place_id': 'ChIJt4p822tLHRUR9_Mz4hg4qQA', 'plus_code': {'compound_code': '3Q2R+HF Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+HF'}, 'rating': 5, 'reference': 'ChIJt4p822tLHRUR9_Mz4hg4qQA', 'scope': 'GOOGLE', 'types': ['electronics_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 36, 'vicinity': 'Chanoch Street 62, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0513199, 'lng': 34.7966304}, 'viewport': {'northeast': {'lat': 32.0527012802915, 'lng': 34.7977243302915}, 'southwest': {'lat': 32.0500033197085, 'lng': 34.7950263697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/school-71.png', 'name': 'בי"ס הגליל', 'opening_hours': {'open_now': True}, 'place_id': 'ChIJAUB2qxRLHRUR-v_rrNM4OO4', 'plus_code': {'compound_code': '3Q2W+GM Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2W+GM'}, 'rating': 4.6, 'reference': 'ChIJAUB2qxRLHRUR-v_rrNM4OO4', 'scope': 'GOOGLE', 'types': ['primary_school', 'school', 'point_of_interest', 'establishment'], 'user_ratings_total': 5, 'vicinity': "Giv'ati Street 4, Tel Aviv-Yafo"}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0434527, 'lng': 34.7930253}, 'viewport': {'northeast': {'lat': 32.04491548029149, 'lng': 34.79437763029149}, 'southwest': {'lat': 32.0422175197085, 'lng': 34.79167966970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'מטרה משרדית', 'opening_hours': {'open_now': False}, 'photos': [{'height': 1683, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/116843049532940032422">מטרה משרדית</a>'], 'photo_reference': 'CkQ0AAAAcFQ16W19myu6eFvA8LZIXf4rltIc1sAnCEdIHsEcvZ9A33QcaKeCODaglBYTBNND5Nq7Cnl3MNEt7G8X4v3n2BIQ_Wv0HeeqFWp0fdvQvDLH-BoUTbQGr3zKFSdvb0a-Wmd9QzGK_xM', 'width': 1000}], 'place_id': 'ChIJK4YSfRBLHRUR8abtfmP3PPY', 'plus_code': {'compound_code': '2QVV+96 Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QVV+96'}, 'rating': 4.6, 'reference': 'ChIJK4YSfRBLHRUR8abtfmP3PPY', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'store', 'establishment'], 'user_ratings_total': 11, 'vicinity': 'Dokhan Street 14, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.051634, 'lng': 34.7916312}, 'viewport': {'northeast': {'lat': 32.0529803302915, 'lng': 34.79294548029149}, 'southwest': {'lat': 32.0502823697085, 'lng': 34.79024751970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/shopping-71.png', 'name': 'לה לה לנד', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1440, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104197150250615112018">עידן אברהמי</a>'], 'photo_reference': 'CkQ0AAAAmVoBo0BsIBYv54Z7teSa6nJUaV0lz0JM7sMcs67X6nXXBWUStkEXG5uWRWWOFwDJ5xdcbZ4pjOhsB3ifcDzJrhIQB1x88Vo84JcjxnK_aXFpIRoUZr70FcccOWc2lFyYN5c0eIjlHug', 'width': 1432}], 'place_id': 'ChIJ5R8PwmtLHRUR9vgFshdgRrU', 'plus_code': {'compound_code': '3Q2R+MM Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q2R+MM'}, 'rating': 4.1, 'reference': 'ChIJ5R8PwmtLHRUR9vgFshdgRrU', 'scope': 'GOOGLE', 'types': ['clothing_store', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 14, 'vicinity': 'Etsel Street 43-47, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0541772, 'lng': 34.7896007}, 'viewport': {'northeast': {'lat': 32.05552868029149, 'lng': 34.7909500302915}, 'southwest': {'lat': 32.0528307197085, 'lng': 34.78825206970851}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'CITY-TOURS חברת הסעות', 'opening_hours': {'open_now': True}, 'photos': [{'height': 1536, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100262791963091125499">CITY-TOURS חברת הסעות</a>'], 'photo_reference': 'CkQ0AAAA9iTQn8b6UGQ6xyJSh9otJyRONa9TNZmlTH3SfqplaQD4xZe4prHL4pEAkvKQZC5QxEgPfHXs9yC0Go558xOqYRIQKvjSV2KVV4TFvkAVisvOYRoU06kiqV72AMpCk9RcnqFvYD5GmSk', 'width': 2048}], 'place_id': 'ChIJF5RIyG5LHRURopQ_l2P1-G0', 'plus_code': {'compound_code': '3Q3Q+MR Tel Aviv-Yafo, Israel', 'global_code': '8G4P3Q3Q+MR'}, 'rating': 5, 'reference': 'ChIJF5RIyG5LHRURopQ_l2P1-G0', 'scope': 'GOOGLE', 'types': ['point_of_interest', 'establishment'], 'user_ratings_total': 14, 'vicinity': 'Hagana Road 57, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.0493398, 'lng': 34.7986866}, 'viewport': {'northeast': {'lat': 32.0506976802915, 'lng': 34.79997263029149}, 'southwest': {'lat': 32.0479997197085, 'lng': 34.79727466970849}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/generic_business-71.png', 'name': 'General Child Health Center', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113313517006366274893">Sigalit Basteker</a>'], 'photo_reference': 'CkQ0AAAAESZMc8-ivEzCXVEGpSYw4zoLX7abWxV1kJSjG8Pf7Ntx92xaZ11h2yp0aRK5M2xB-SU0WB11QSvi4UssnmjDBhIQsqKy7xgzAJq2EPfM8ERIMRoUEOTraGLhUpvvCSQ-jO3bucG5rsM', 'width': 2250}], 'place_id': 'ChIJRT9cixRLHRURAoL3cSXy2Oc', 'plus_code': {'compound_code': '2QXX+PF Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXX+PF'}, 'rating': 1.8, 'reference': 'ChIJRT9cixRLHRURAoL3cSXy2Oc', 'scope': 'GOOGLE', 'types': ['health', 'point_of_interest', 'establishment'], 'user_ratings_total': 5, 'vicinity': 'Hartuv Street 5, Tel Aviv-Yafo'}, {'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 32.049287, 'lng': 34.7987451}, 'viewport': {'northeast': {'lat': 32.0506496302915, 'lng': 34.7999976302915}, 'southwest': {'lat': 32.0479516697085, 'lng': 34.7972996697085}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/pharmacy_cross-71.png', 'name': 'מרפאת נטקה - קופ"ח כללית', 'opening_hours': {'open_now': True}, 'photos': [{'height': 4000, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/113313517006366274893">Sigalit Basteker</a>'], 'photo_reference': 'CkQ0AAAAbWYCqFdEr0y0-AJ5aC4YkQ1ABdSiKU3r-LL2zyHFTtmJsqNN54xzUAMgosRBwPGT-AuK3x3fvBAdchQgPhPPRhIQqciqZ2TXAVp14qVNrmjE2xoUmFvI-73Qm1w5dOd5CmKEGCi8kho', 'width': 2250}], 'place_id': 'ChIJRT9cixRLHRURDNf_e1V5aC0', 'plus_code': {'compound_code': '2QXX+PF Tel Aviv-Yafo, Israel', 'global_code': '8G4P2QXX+PF'}, 'rating': 4, 'reference': 'ChIJRT9cixRLHRURDNf_e1V5aC0', 'scope': 'GOOGLE', 'types': ['pharmacy', 'health', 'point_of_interest', 'store', 'establishment'], 'user_ratings_total': 17, 'vicinity': 'Hartuv Street 5, Tel Aviv-Yafo'}]}
for x in data:
    print(x)

32.047691, 34.793865


In [10]:
for coordinate in data:
    my_data = data[coordinate]
    name = my_data.index("name")
    my_coordinate = pd.DataFrame(data[coordinate])
    

In [11]:
my_coordinate

,geometry,icon,name,photos,place_id,reference,scope,types,vicinity,business_status,opening_hours,plus_code,permanently_closed,rating,user_ratings_total,price_level
0,"{'location': {'lat': 32.0852999, 'lng': 34.781...",https://maps.gstatic.com/mapfiles/place_api/ic...,Tel Aviv-Yafo,"[{'height': 3048, 'html_attributions': ['<a hr...",ChIJH3w7GaZMHRURkD-WwKJy-8E,ChIJH3w7GaZMHRURkD-WwKJy-8E,GOOGLE,"[locality, political]",Tel Aviv-Yafo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'location': {'lat': 32.05023600000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,Peleg Hiper Pharm,NaN,ChIJFbUdNWpLHRUR-wO6GuhWP58,ChIJFbUdNWpLHRUR-wO6GuhWP58,GOOGLE,"[pharmacy, health, point_of_interest, store, e...",Tel Aviv-Yafo,OPERATIONAL,{'open_now': True},"{'compound_code': '3Q2R+3P Tel Aviv-Yafo, Isra...",NaN,NaN,NaN,NaN
2,"{'location': {'lat': 32.045517, 'lng': 34.8034...",https://maps.gstatic.com/mapfiles/place_api/ic...,Chen's Holiday Home: Women Only,"[{'height': 4160, 'html_attributions': ['<a hr...",ChIJ44RNRxZLHRUR8wC2sAnAYC4,ChIJ44RNRxZLHRUR8wC2sAnAYC4,GOOGLE,"[lodging, point_of_interest, establishment]","HaRav Yitshak Taib Street 34, Tel Aviv-Yafo",CLOSED_TEMPORARILY,NaN,"{'compound_code': '2RW3+69 Tel Aviv-Yafo, Isra...",True,4.1,7.0,NaN
3,"{'location': {'lat': 32.0532695, 'lng': 34.790...",https://maps.gstatic.com/mapfiles/place_api/ic...,Naama spices,"[{'height': 3984, 'html_attributions': ['<a hr...",ChIJ1x0rB2xLHRURJGZw0Y3RoRc,ChIJ1x0rB2xLHRURJGZw0Y3RoRc,GOOGLE,"[food, point_of_interest, store, establishment]","Mevaser Street 4, Tel Aviv-Yafo",OPERATIONAL,{'open_now': True},"{'compound_code': '3Q3R+89 Tel Aviv-Yafo, Isra...",NaN,4.7,204.0,NaN
4,"{'location': {'lat': 32.0454216, 'lng': 34.794...",https://maps.gstatic.com/mapfiles/place_api/ic...,פון קאבר מחסן יבואן אביזרים לסלולר בסיטונאות ו...,"[{'height': 1536, 'html_attributions': ['<a hr...",ChIJFYeUaUBLHRURl8jiRqyltiY,ChIJFYeUaUBLHRURl8jiRqyltiY,GOOGLE,"[electronics_store, point_of_interest, store, ...","Malkosh Street 11, Tel Aviv-Yafo",OPERATIONAL,{'open_now': True},"{'compound_code': '2QWV+5P Tel Aviv-Yafo, Isra...",NaN,4.8,182.0,NaN
5,"{'location': {'lat': 32.0481402, 'lng': 34.793...",https://maps.gstatic.com/mapfiles/place_api/ic...,Big-Bag,NaN,ChIJ8_ewjmpLHRURnkvIi_dDF80,ChIJ8_ewjmpLHRURnkvIi_dDF80,GOOGLE,"[furniture_store, home_goods_store, point_of_i...","Bat Sheva Katzenelson Street 4, Tel Aviv-Yafo",OPERATIONAL,{'open_now': False},"{'compound_code': '2QXV+7F Tel Aviv-Yafo, Isra...",NaN,5.0,1.0,NaN
6,"{'location': {'lat': 32.0476349, 'lng': 34.798...",https://maps.gstatic.com/mapfiles/place_api/ic...,Thimble Publishing Ltd.,"[{'height': 2272, 'html_attributions': ['<a hr...",ChIJH7T057hMHRURAFsRitKEejU,ChIJH7T057hMHRURAFsRitKEejU,GOOGLE,"[library, book_store, point_of_interest, store...","Hartuv Street 26, Tel Aviv-Yafo",OPERATIONAL,{'open_now': False},"{'compound_code': '2QXX+36 Tel Aviv-Yafo, Isra...",NaN,5.0,2.0,NaN
7,"{'location': {'lat': 32.0506907, 'lng': 34.794...",https://maps.gstatic.com/mapfiles/place_api/ic...,ARIEL HARLEV,NaN,ChIJs35eP7FLHRUR7uEyFjWClA4,ChIJs35eP7FLHRUR7uEyFjWClA4,GOOGLE,"[health, point_of_interest, establishment]","Bat Sheva Katzenelson Street 94, Giv'atayim",OPERATIONAL,NaN,"{'compound_code': '3Q2V+7M Giv'atayim, Israel'...",NaN,5.0,1.0,NaN
8,"{'location': {'lat': 32.0440165, 'lng': 34.795...",https://maps.gstatic.com/mapfiles/place_api/ic...,אילן הובלות,"[{'height': 960, 'html_attributions': ['<a hre...",ChIJSaxyiEBLHRURIBkQzRvLFkU,ChIJSaxyiEBLHRURIBkQzRvLFkU,GOOGLE,"[moving_company, storage, point_of_interest, e...","Shtulim Street 50, Tel Aviv-Yafo",OPERATIONAL,{'open_now': True},"{'compound_code': '2QVW+J5 Tel Aviv-Yafo, Isra...",NaN,4.3,16.0,NaN
9,"{'location': {'lat': 32.049792, 'lng': 34.7921...",https://maps.gstatic.com/mapfiles/place_api/ic...,Avazi,"[{'height': 1080, 'html_attributions': ['<a hr...",ChIJW6tZPnxLHRURHQzVXXo51eg,ChIJW6tZPnxLHRURHQzVXXo51eg,GOOGLE,"[restaurant, food, point_of_interest, establis...","Etsel Stree